In [1]:
import numpy as np
import os
from  natsort import natsorted
import imageio
import re
import time
import keras
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization, Input, GRU, Reshape
from keras.layers import Conv2D, MaxPooling2D, Bidirectional, Permute, LSTM
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.layers.merge import add, concatenate
from keras import backend as K
from keras.models import load_model, Model
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score, accuracy_score
import matplotlib.pyplot as plt
import itertools

Using TensorFlow backend.


In [2]:
NAME = 'ALPR_CNN'
data_dir = 'Data'
data_dir_2 = 'Data/dataset_1'
model_dir = 'Models'

In [3]:
def prepare_dataset(data_dir, folder_name):
    try:
        print('Loading numpy')
        X = np.load(os.path.join(data_dir,'X_{}.npy'.format(folder_name)))
        y = np.load(os.path.join(data_dir,'y_{}.npy'.format(folder_name)))

    except:
        print('Loading images')
        image_list = []
        labels = []
        pictures_dir = os.path.join(data_dir, folder_name)
        names = [ d for d in os.listdir( pictures_dir ) if d.endswith( '.jpg') ]
        names = natsorted(names)
        for image in names:
            image_list.append(imageio.imread(os.path.join(pictures_dir, image)))
            label = re.split('[._]', image)
            labels.append(label[0][2:])
            print(label[0][2:])
        X = np.stack(image_list, axis=0)
        y = np.array(labels)
        np.save(os.path.join(data_dir,'X_{}'.format(folder_name)),X)
        np.save(os.path.join(data_dir,'y_{}'.format(folder_name)),y)
    return X,y


def prepare_dataset_2(data_dir, folder_name):
    try:
        print('Loading numpy')
        X = np.load(os.path.join(data_dir,'X_{}.npy'.format(folder_name)))
        y = np.load(os.path.join(data_dir,'y_{}.npy'.format(folder_name)))

    except:
        print('Loading images')
        image_list = []
        labels = []
        pictures_dir = os.path.join(data_dir, folder_name)
        names = [ d for d in os.listdir( pictures_dir ) if d.endswith( '.jpg') ]
        names = natsorted(names)
        for image in names:
            image_list.append(imageio.imread(os.path.join(pictures_dir, image)))
            label = re.split('[._]', image)
            labels.append(label[1])
            print(label[1])
        X = np.stack(image_list, axis=0)
        y = np.array(labels)
        np.save(os.path.join(data_dir,'X_{}'.format(folder_name)),X)
        np.save(os.path.join(data_dir,'y_{}'.format(folder_name)),y)
    return X,y

In [4]:
X1,y1 = prepare_dataset(data_dir, 'resized')
print(X1.shape)
print(y1.shape)

Loading numpy
(3921, 100, 160, 3)
(3921,)


In [5]:
X2,y2 = prepare_dataset_2(data_dir_2, 'Resized')
print(X2.shape)
print(y2.shape)

Loading numpy
(300, 100, 160, 3)
(300,)


In [6]:
X = np.concatenate((X1,X2), axis=0)
y = np.concatenate((y1,y2), axis=0)
print(X.shape)
print(y.shape)

(4221, 100, 160, 3)
(4221,)


In [7]:
X_test,y_test = prepare_dataset_2(data_dir_2, 'Resized_test')
print(X_test.shape)
print(y_test.shape)

Loading numpy
(1749, 100, 160, 3)
(1749,)


In [8]:
np.random.seed(42)
indices = np.arange(X.shape[0])
np.random.shuffle(indices)

X = X[indices]
y = y[indices]

In [9]:
z = 3800
X_train = X[:z]
X_val = X[z:]
y_train = y[:z]
y_val = y[z:]

In [10]:
regex = r'^[A-Z0-9 ]+$'
alphabet = u'ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789 '

In [11]:
# Translation of characters to unique integer values
def text_to_labels(text):
    ret = []
    for char in text:
        ret.append(alphabet.find(char))
    return ret


# Reverse translation of numerical classes back to characters
def labels_to_text(labels):
    ret = []
    for c in labels:
        if c == len(alphabet):  # CTC Blank
            ret.append("")
        else:
            ret.append(alphabet[c])
    return "".join(ret)


# only a-z and space..probably not to difficult
# to expand to uppercase and symbols

def is_valid_str(in_str):
    search = re.compile(regex, re.UNICODE).search
    return bool(search(in_str))

In [28]:
#Removing 7 label plates
mylen = np.vectorize(len)

count = mylen(y_train)
X_train = np.delete(X_train, np.where(count>6)[0], axis=0 )
y_train = np.delete(y_train, np.where(count>6)[0], axis=0 )

count = mylen(y_val)
X_val = np.delete(X_val, np.where(count>6)[0], axis=0 )
y_val = np.delete(y_val, np.where(count>6)[0], axis=0 )

count = mylen(y_test)
X_test = np.delete(X_test, np.where(count!=6)[0], axis=0 )
y_test = np.delete(y_test, np.where(count!=6)[0], axis=0 )

In [13]:
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)
print(X_test.shape)
print(y_test.shape)

(3796, 100, 160, 3)
(3796,)
(419, 100, 160, 3)
(419,)
(1748, 100, 160, 3)
(1748,)


In [14]:
def code_labels(labels):
    y_coded = []
    for l in labels:
        y_coded.append(text_to_labels(l))
    return np.array(y_coded)

In [29]:
y_train_coded = code_labels(y_train)
y_val_coded = code_labels(y_val)
y_test_coded = code_labels(y_test)
print(y_train_coded.shape)
print(y_val_coded.shape)
print(y_test_coded.shape)

(3796, 6)
(419, 6)
(1740, 6)


In [31]:
num_classes = 36
#Labels to binary
y_train_0 = keras.utils.to_categorical(y_train_coded[:,0],num_classes)
y_train_1 = keras.utils.to_categorical(y_train_coded[:,1],num_classes)
y_train_2 = keras.utils.to_categorical(y_train_coded[:,2],num_classes)
y_train_3 = keras.utils.to_categorical(y_train_coded[:,3],num_classes)
y_train_4 = keras.utils.to_categorical(y_train_coded[:,4],num_classes)
y_train_5 = keras.utils.to_categorical(y_train_coded[:,5],num_classes)

y_val_0 = keras.utils.to_categorical(y_val_coded[:,0],num_classes)
y_val_1 = keras.utils.to_categorical(y_val_coded[:,1],num_classes)
y_val_2 = keras.utils.to_categorical(y_val_coded[:,2],num_classes)
y_val_3 = keras.utils.to_categorical(y_val_coded[:,3],num_classes)
y_val_4 = keras.utils.to_categorical(y_val_coded[:,4],num_classes)
y_val_5 = keras.utils.to_categorical(y_val_coded[:,5],num_classes)

y_test_0 = keras.utils.to_categorical(y_test_coded[:,0],num_classes)
y_test_1 = keras.utils.to_categorical(y_test_coded[:,1],num_classes)
y_test_2 = keras.utils.to_categorical(y_test_coded[:,2],num_classes)
y_test_3 = keras.utils.to_categorical(y_test_coded[:,3],num_classes)
y_test_4 = keras.utils.to_categorical(y_test_coded[:,4],num_classes)
y_test_5 = keras.utils.to_categorical(y_test_coded[:,5],num_classes)


In [30]:
from collections import Counter
Counter(count)

Counter({3: 1, 4: 5, 5: 2, 6: 1740})

## Keras Model

In [29]:
img_w = 100
# Input Parameters
img_h = 160
# Network parameters
conv_filters = 64
kernel_size = (3, 3)
pool_size = 2
time_dense_size = 32
rnn_size = 64
minibatch_size = 128
unique_tokens = 37

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_w, img_h)
else:
    input_shape = (img_w, img_h, 3)


In [32]:
num_feat_map = 64
p = 0.3

In [33]:
input_data = Input(name='the_input', shape=input_shape, dtype='float32')
x = Conv2D(32, kernel_size=(3, 3),
               activation='relu',
               input_shape=input_shape,
               padding='same', name='Conv_1')(input_data)
x = MaxPooling2D(pool_size=(2, 2), name='Max_pool_1')(x)
x = BatchNormalization(name='Bn_1')(x)
x = Dropout(p, name='Drop_1')(x)
x = Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same', name='Conv_2')(x)
#x = MaxPooling2D(pool_size=(2, 2), name='Max_pool_2')(x)
x = BatchNormalization(name='Bn_2')(x)
x = Dropout(p, name='Drop_2')(x)
x = Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same', name='Conv_3')(x)
x = MaxPooling2D(pool_size=(2, 2), name='Max_pool_3')(x)
x = BatchNormalization(name='Bn_3')(x)
x = Dropout(p, name='Drop_3')(x)
x = Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same', name='Conv_4')(x)
#x = MaxPooling2D(pool_size=(2, 2), name='Max_pool_4')(x)
x = BatchNormalization(name='Bn_4')(x)
x = Dropout(p, name='Drop_4')(x)
x = Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same', name='Conv_5')(x)
x = MaxPooling2D(pool_size=(2, 2), name='Max_pool_5')(x)
x = BatchNormalization(name='Bn_5')(x)
x = Dropout(p, name='Drop_5')(x)
x = Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same', name='Conv_6')(x)
#x = MaxPooling2D(pool_size=(2, 2), name='Max_pool_6')(x)
x = BatchNormalization(name='Bn_6')(x)
x = Dropout(p, name='Drop_6')(x)
x = Conv2D(16, kernel_size=(3, 3), activation='relu', padding='same', name='Conv_7')(x)
#x = MaxPooling2D(pool_size=(2, 2), name='Max_pool_7')(x)
x = BatchNormalization(name='Bn_7')(x)
x = Dropout(p, name='Drop_7')(x)
# model.add(Conv2D(num_feat_map, kernel_size=(3, 3), activation='relu', padding='same', name='Conv_3'))
# model.add(BatchNormalization(name='Bn_3'))
# model.add(MaxPooling2D(pool_size=(2, 2), name='Max_pool_3'))
# model.add(Dropout(p, name='Drop_3'))
# x = Permute((2, 1, 3), name='Permute_1')(x)  # for swap-dimension
# x = Reshape((-1, num_feat_map * (img_w // (pool_size ** 2))), name='Reshape_1')(x)
# x = Dense(time_dense_size, activation='relu', name='dense1')(x)
# x = Bidirectional(LSTM(32, return_sequences=True, stateful=False, name='Lstm_1'))(x)
# x = BatchNormalization(name='Bn_4')(x)
# x = Bidirectional(LSTM(32, return_sequences=True, stateful=False, name='Lstm_1'))(x)
# x = BatchNormalization(name='Bn_5')(x)
# x = Dropout(p, name='Drop_4')(x)
#x = Dense(128, activation='relu', name='dense_2')(x)
x = Flatten()(x)
x = Dense(128, activation='relu', name='dense_1')(x)
x = BatchNormalization(name='Bn_8')(x)
x = Dropout(p, name='Drop_8')(x)

#output
d1 = Dense(num_classes, activation='softmax', name='dense_out_1')(x)
d2 = Dense(num_classes, activation='softmax', name='dense_out_2' )(x)
d3 = Dense(num_classes, activation='softmax', name='dense_out_3')(x)
d4 = Dense(num_classes, activation='softmax', name='dense_out_4')(x)
d5 = Dense(num_classes, activation='softmax', name='dense_out_5')(x)
d6 = Dense(num_classes, activation='softmax', name='dense_out_6')(x)
model = Model(input_data, [d1,d2,d3,d4,d5,d6])


model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
the_input (InputLayer)          (None, 100, 160, 3)  0                                            
__________________________________________________________________________________________________
Conv_1 (Conv2D)                 (None, 100, 160, 32) 896         the_input[0][0]                  
__________________________________________________________________________________________________
Max_pool_1 (MaxPooling2D)       (None, 50, 80, 32)   0           Conv_1[0][0]                     
__________________________________________________________________________________________________
Bn_1 (BatchNormalization)       (None, 50, 80, 32)   128         Max_pool_1[0][0]                 
__________________________________________________________________________________________________
Drop_1 (Dr

In [34]:
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

batch_size = 50
epochs = 100


In [36]:
model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer='adam',
                  metrics=['accuracy'])
t = int(time.time())

# checkpoint
chk_path = os.path.join(model_dir, 'best_{}_{}'.format(NAME,t))
checkpoint = ModelCheckpoint(chk_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
tensorboard = TensorBoard(log_dir="logs/{}_{}".format(NAME,t))
callbacks_list = [checkpoint, tensorboard]

history = model.fit(X_train, [y_train_0, y_train_1, y_train_2, y_train_3, y_train_4, y_train_5],
            batch_size=batch_size,
            epochs=epochs,
            verbose=1,
            shuffle=True,
            validation_data=(X_val, [y_val_0, y_val_1, y_val_2, y_val_3, y_val_4, y_val_5]),
            callbacks=callbacks_list)

#Saving the model
model.save(os.path.join(model_dir, 'final_{}_{}'.format(NAME,t)))

Train on 3796 samples, validate on 419 samples
Epoch 1/100
3796/3796 [==============================] - 20s 5ms/step - loss: 19.4001 - dense_out_1_loss: 3.4323 - dense_out_2_loss: 3.4264 - dense_out_3_loss: 3.2138 - dense_out_4_loss: 2.9641 - dense_out_5_loss: 3.1304 - dense_out_6_loss: 3.2331 - dense_out_1_acc: 0.1955 - dense_out_2_acc: 0.1891 - dense_out_3_acc: 0.2366 - dense_out_4_acc: 0.2924 - dense_out_5_acc: 0.2677 - dense_out_6_acc: 0.2479 - val_loss: 22.9489 - val_dense_out_1_loss: 3.8299 - val_dense_out_2_loss: 4.0741 - val_dense_out_3_loss: 3.6449 - val_dense_out_4_loss: 3.8155 - val_dense_out_5_loss: 3.3229 - val_dense_out_6_loss: 4.2616 - val_dense_out_1_acc: 0.2625 - val_dense_out_2_acc: 0.2029 - val_dense_out_3_acc: 0.3246 - val_dense_out_4_acc: 0.3866 - val_dense_out_5_acc: 0.3628 - val_dense_out_6_acc: 0.3150
Epoch 2/100


c:\programdata\anaconda3\envs\tensorflow\lib\site-packages\keras\callbacks.py:432: RuntimeWarning: Can save best model only with val_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


3796/3796 [==============================] - 12s 3ms/step - loss: 12.7273 - dense_out_1_loss: 2.4788 - dense_out_2_loss: 2.5656 - dense_out_3_loss: 2.0915 - dense_out_4_loss: 1.6339 - dense_out_5_loss: 1.8408 - dense_out_6_loss: 2.1167 - dense_out_1_acc: 0.3854 - dense_out_2_acc: 0.3514 - dense_out_3_acc: 0.4747 - dense_out_4_acc: 0.5558 - dense_out_5_acc: 0.5337 - dense_out_6_acc: 0.4615 - val_loss: 10.9314 - val_dense_out_1_loss: 2.0356 - val_dense_out_2_loss: 2.1550 - val_dense_out_3_loss: 1.7859 - val_dense_out_4_loss: 1.5619 - val_dense_out_5_loss: 1.5531 - val_dense_out_6_loss: 1.8399 - val_dense_out_1_acc: 0.4940 - val_dense_out_2_acc: 0.4415 - val_dense_out_3_acc: 0.5609 - val_dense_out_4_acc: 0.6205 - val_dense_out_5_acc: 0.5943 - val_dense_out_6_acc: 0.5346
Epoch 3/100
3796/3796 [==============================] - 12s 3ms/step - loss: 9.6371 - dense_out_1_loss: 1.8551 - dense_out_2_loss: 2.0148 - dense_out_3_loss: 1.6269 - dense_out_4_loss: 1.1821 - dense_out_5_loss: 1.3756 - 

3796/3796 [==============================] - 12s 3ms/step - loss: 2.8521 - dense_out_1_loss: 0.5646 - dense_out_2_loss: 0.5824 - dense_out_3_loss: 0.4529 - dense_out_4_loss: 0.3278 - dense_out_5_loss: 0.4120 - dense_out_6_loss: 0.5124 - dense_out_1_acc: 0.8427 - dense_out_2_acc: 0.8401 - dense_out_3_acc: 0.8680 - dense_out_4_acc: 0.9086 - dense_out_5_acc: 0.8846 - dense_out_6_acc: 0.8546 - val_loss: 3.0498 - val_dense_out_1_loss: 0.6015 - val_dense_out_2_loss: 0.6728 - val_dense_out_3_loss: 0.4598 - val_dense_out_4_loss: 0.3769 - val_dense_out_5_loss: 0.4194 - val_dense_out_6_loss: 0.5195 - val_dense_out_1_acc: 0.8234 - val_dense_out_2_acc: 0.8043 - val_dense_out_3_acc: 0.8616 - val_dense_out_4_acc: 0.9117 - val_dense_out_5_acc: 0.8878 - val_dense_out_6_acc: 0.8568
Epoch 13/100
3796/3796 [==============================] - 12s 3ms/step - loss: 2.5537 - dense_out_1_loss: 0.5051 - dense_out_2_loss: 0.5259 - dense_out_3_loss: 0.4101 - dense_out_4_loss: 0.2787 - dense_out_5_loss: 0.3776 - d

3796/3796 [==============================] - 13s 3ms/step - loss: 1.3124 - dense_out_1_loss: 0.2678 - dense_out_2_loss: 0.2639 - dense_out_3_loss: 0.2052 - dense_out_4_loss: 0.1548 - dense_out_5_loss: 0.1895 - dense_out_6_loss: 0.2313 - dense_out_1_acc: 0.9210 - dense_out_2_acc: 0.9233 - dense_out_3_acc: 0.9455 - dense_out_4_acc: 0.9552 - dense_out_5_acc: 0.9402 - dense_out_6_acc: 0.9312 - val_loss: 2.2402 - val_dense_out_1_loss: 0.4340 - val_dense_out_2_loss: 0.4858 - val_dense_out_3_loss: 0.3369 - val_dense_out_4_loss: 0.3185 - val_dense_out_5_loss: 0.2980 - val_dense_out_6_loss: 0.3670 - val_dense_out_1_acc: 0.8711 - val_dense_out_2_acc: 0.8640 - val_dense_out_3_acc: 0.9165 - val_dense_out_4_acc: 0.9451 - val_dense_out_5_acc: 0.9189 - val_dense_out_6_acc: 0.9021
Epoch 23/100
3796/3796 [==============================] - 12s 3ms/step - loss: 1.2818 - dense_out_1_loss: 0.2534 - dense_out_2_loss: 0.2644 - dense_out_3_loss: 0.2010 - dense_out_4_loss: 0.1508 - dense_out_5_loss: 0.1874 - d

...

3796/3796 [==============================] - 13s 3ms/step - loss: 0.5925 - dense_out_1_loss: 0.1156 - dense_out_2_loss: 0.1195 - dense_out_3_loss: 0.0905 - dense_out_4_loss: 0.0740 - dense_out_5_loss: 0.0879 - dense_out_6_loss: 0.1050 - dense_out_1_acc: 0.9660 - dense_out_2_acc: 0.9655 - dense_out_3_acc: 0.9718 - dense_out_4_acc: 0.9752 - dense_out_5_acc: 0.9747 - dense_out_6_acc: 0.9673 - val_loss: 1.8272 - val_dense_out_1_loss: 0.3230 - val_dense_out_2_loss: 0.3650 - val_dense_out_3_loss: 0.3070 - val_dense_out_4_loss: 0.3202 - val_dense_out_5_loss: 0.2330 - val_dense_out_6_loss: 0.2789 - val_dense_out_1_acc: 0.9093 - val_dense_out_2_acc: 0.9093 - val_dense_out_3_acc: 0.9403 - val_dense_out_4_acc: 0.9451 - val_dense_out_5_acc: 0.9523 - val_dense_out_6_acc: 0.9212
Epoch 53/100
3796/3796 [==============================] - 12s 3ms/step - loss: 0.5828 - dense_out_1_loss: 0.1149 - dense_out_2_loss: 0.1212 - dense_out_3_loss: 0.0896 - dense_out_4_loss: 0.0649 - dense_out_5_loss: 0.0888 - d

3796/3796 [==============================] - 12s 3ms/step - loss: 0.4383 - dense_out_1_loss: 0.0816 - dense_out_2_loss: 0.0770 - dense_out_3_loss: 0.0795 - dense_out_4_loss: 0.0514 - dense_out_5_loss: 0.0644 - dense_out_6_loss: 0.0843 - dense_out_1_acc: 0.9737 - dense_out_2_acc: 0.9760 - dense_out_3_acc: 0.9739 - dense_out_4_acc: 0.9829 - dense_out_5_acc: 0.9797 - dense_out_6_acc: 0.9758 - val_loss: 1.8360 - val_dense_out_1_loss: 0.3318 - val_dense_out_2_loss: 0.3583 - val_dense_out_3_loss: 0.3116 - val_dense_out_4_loss: 0.3319 - val_dense_out_5_loss: 0.2080 - val_dense_out_6_loss: 0.2944 - val_dense_out_1_acc: 0.9069 - val_dense_out_2_acc: 0.9141 - val_dense_out_3_acc: 0.9332 - val_dense_out_4_acc: 0.9451 - val_dense_out_5_acc: 0.9570 - val_dense_out_6_acc: 0.9236
Epoch 63/100
3796/3796 [==============================] - 12s 3ms/step - loss: 0.4630 - dense_out_1_loss: 0.0866 - dense_out_2_loss: 0.0903 - dense_out_3_loss: 0.0797 - dense_out_4_loss: 0.0541 - dense_out_5_loss: 0.0643 - d

3796/3796 [==============================] - 12s 3ms/step - loss: 0.4513 - dense_out_1_loss: 0.0901 - dense_out_2_loss: 0.0856 - dense_out_3_loss: 0.0769 - dense_out_4_loss: 0.0504 - dense_out_5_loss: 0.0687 - dense_out_6_loss: 0.0796 - dense_out_1_acc: 0.9721 - dense_out_2_acc: 0.9694 - dense_out_3_acc: 0.9744 - dense_out_4_acc: 0.9850 - dense_out_5_acc: 0.9800 - dense_out_6_acc: 0.9744 - val_loss: 1.7943 - val_dense_out_1_loss: 0.3262 - val_dense_out_2_loss: 0.3829 - val_dense_out_3_loss: 0.2979 - val_dense_out_4_loss: 0.3236 - val_dense_out_5_loss: 0.2184 - val_dense_out_6_loss: 0.2453 - val_dense_out_1_acc: 0.9117 - val_dense_out_2_acc: 0.9045 - val_dense_out_3_acc: 0.9308 - val_dense_out_4_acc: 0.9499 - val_dense_out_5_acc: 0.9523 - val_dense_out_6_acc: 0.9427
Epoch 73/100
3796/3796 [==============================] - 13s 3ms/step - loss: 0.4192 - dense_out_1_loss: 0.0764 - dense_out_2_loss: 0.0813 - dense_out_3_loss: 0.0708 - dense_out_4_loss: 0.0542 - dense_out_5_loss: 0.0618 - d

3796/3796 [==============================] - 12s 3ms/step - loss: 0.4039 - dense_out_1_loss: 0.0798 - dense_out_2_loss: 0.0701 - dense_out_3_loss: 0.0700 - dense_out_4_loss: 0.0522 - dense_out_5_loss: 0.0602 - dense_out_6_loss: 0.0716 - dense_out_1_acc: 0.9758 - dense_out_2_acc: 0.9784 - dense_out_3_acc: 0.9776 - dense_out_4_acc: 0.9829 - dense_out_5_acc: 0.9805 - dense_out_6_acc: 0.9781 - val_loss: 1.8599 - val_dense_out_1_loss: 0.3312 - val_dense_out_2_loss: 0.4027 - val_dense_out_3_loss: 0.2845 - val_dense_out_4_loss: 0.3291 - val_dense_out_5_loss: 0.2444 - val_dense_out_6_loss: 0.2680 - val_dense_out_1_acc: 0.9189 - val_dense_out_2_acc: 0.9165 - val_dense_out_3_acc: 0.9284 - val_dense_out_4_acc: 0.9475 - val_dense_out_5_acc: 0.9475 - val_dense_out_6_acc: 0.9403
Epoch 83/100
3796/3796 [==============================] - 12s 3ms/step - loss: 0.3869 - dense_out_1_loss: 0.0780 - dense_out_2_loss: 0.0733 - dense_out_3_loss: 0.0621 - dense_out_4_loss: 0.0503 - dense_out_5_loss: 0.0485 - d

3796/3796 [==============================] - 12s 3ms/step - loss: 0.3910 - dense_out_1_loss: 0.0763 - dense_out_2_loss: 0.0716 - dense_out_3_loss: 0.0621 - dense_out_4_loss: 0.0451 - dense_out_5_loss: 0.0634 - dense_out_6_loss: 0.0725 - dense_out_1_acc: 0.9734 - dense_out_2_acc: 0.9779 - dense_out_3_acc: 0.9789 - dense_out_4_acc: 0.9868 - dense_out_5_acc: 0.9802 - dense_out_6_acc: 0.9773 - val_loss: 1.7550 - val_dense_out_1_loss: 0.3554 - val_dense_out_2_loss: 0.3316 - val_dense_out_3_loss: 0.2894 - val_dense_out_4_loss: 0.2936 - val_dense_out_5_loss: 0.2414 - val_dense_out_6_loss: 0.2435 - val_dense_out_1_acc: 0.9021 - val_dense_out_2_acc: 0.9165 - val_dense_out_3_acc: 0.9308 - val_dense_out_4_acc: 0.9403 - val_dense_out_5_acc: 0.9427 - val_dense_out_6_acc: 0.9332
Epoch 93/100
3796/3796 [==============================] - 12s 3ms/step - loss: 0.3852 - dense_out_1_loss: 0.0821 - dense_out_2_loss: 0.0708 - dense_out_3_loss: 0.0589 - dense_out_4_loss: 0.0426 - dense_out_5_loss: 0.0586 - d

In [4]:
import tensorflow as tf

In [62]:
model = load_model('Models/final_ALPR_CNN_1552982985')

In [63]:
(y_pred_0, y_pred_1, y_pred_2, y_pred_3, y_pred_4, y_pred_5) = model.predict(X_test)

In [64]:
y_pred_0 = np.argmax(y_pred_0, axis=1)
y_pred_1 = np.argmax(y_pred_1, axis=1)
y_pred_2 = np.argmax(y_pred_2, axis=1)
y_pred_3 = np.argmax(y_pred_3, axis=1)
y_pred_4 = np.argmax(y_pred_4, axis=1)
y_pred_5 = np.argmax(y_pred_5, axis=1)

In [67]:
acc = 0
for i,code in enumerate(y_test_coded):
    pred = labels_to_text([y_pred_0[i],y_pred_1[i],y_pred_2[i],y_pred_3[i],y_pred_4[i],y_pred_5[i]])
    true = labels_to_text(code)
    if(pred == true):
        acc = acc+1
print(acc)

255


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

cm = confusion_matrix(y_test_coded[:,0],y_pred_0)
cm = cm/cm.sum(axis=0)
df_cm = pd.DataFrame(cm, index = [i for i in ['Left','Right','Legs','Tongue']],
                  columns = [i for i in ['Left','Right','Legs','Tongue']])
plt.figure(figsize = (10,7))
sn.heatmap(df_cm, annot=True)

In [68]:
from sklearn.metrics import accuracy_score

accuracy_0 = accuracy_score(y_test_coded[:,0],y_pred_0)
accuracy_1 = accuracy_score(y_test_coded[:,1],y_pred_1)
accuracy_2 = accuracy_score(y_test_coded[:,2],y_pred_2)
accuracy_3 = accuracy_score(y_test_coded[:,3],y_pred_3)
accuracy_4 = accuracy_score(y_test_coded[:,4],y_pred_4)
accuracy_5 = accuracy_score(y_test_coded[:,5],y_pred_5)
print(accuracy_0)
print(accuracy_1)
print(accuracy_2)
print(accuracy_3)
print(accuracy_4)
print(accuracy_5)

0.5287356321839081
0.596551724137931
0.8563218390804598
0.9011494252873563
0.6936781609195403
0.6252873563218391
